# Project A — End‑to‑End Implementation Guide (Beginner Friendly)

Welcome! This notebook explains **how to run and understand the entire Project A data pipeline** from Bronze → Silver → Gold. It is written for beginners, with step‑by‑step instructions, commands, and explanations.

**What you will learn:**
- How the data flows through the pipeline
- How to set up your environment
- How to run each stage safely
- How to troubleshoot common issues
- How to explain the project in interviews


## 1. Big‑Picture Architecture (Plain English)

Project A is a **data engineering pipeline** with three main layers:

1. **Bronze (Raw Ingest)**
   - Raw data is ingested from multiple sources (CRM, Snowflake, Redshift, FX JSON, Kafka simulations, etc.).
   - Minimal cleaning; keep it close to source.

2. **Silver (Clean + Standardized)**
   - Data is cleaned, normalized, and deduplicated.
   - Business rules and data contracts are applied.

3. **Gold (Analytics‑Ready)**
   - Dimensional models (facts/dimensions) and analytics tables are built.
   - Optimized for reporting and downstream BI.

**Storage Formats:**
- Local runs can use **Parquet** or **Delta**.
- Iceberg is supported for ACID + time travel.


## 2. Pipeline Flow Diagram

```mermaid
flowchart LR
    A[CRM] --> B[Bronze]
    C[Snowflake] --> B[Bronze]
    D[Redshift] --> B[Bronze]
    E[FX JSON] --> B[Bronze]
    F[Kafka Sim] --> B[Bronze]

    B --> S[Silver]
    S --> G[Gold]
    G --> BI[Analytics / BI]
```


## 3. System Architecture Diagram

```mermaid
flowchart TB
    subgraph Orchestration
        Airflow
    end

    subgraph Compute
        Spark
    end

    subgraph Storage
        S3_or_Local[(S3 or Local Files)]
        Iceberg[(Iceberg / Delta / Parquet)]
    end

    subgraph Governance
        DQ[Data Quality Gates]
        Lineage[Lineage + Metrics]
    end

    Airflow --> Spark
    Spark --> Iceberg
    Iceberg --> S3_or_Local
    Spark --> DQ
    Spark --> Lineage
```


## 4. Prerequisites

Make sure you have:
- **Python 3.10+**
- **Java 11 or 17** (required by Spark)
- **Spark 3.5.x** (bundled via PySpark)
- Optional: **Docker** (for Airflow), **AWS CLI**

If you are unsure, run these commands:
```bash
python --version
java -version
```



## 5. Setup: Virtual Environment + Dependencies

We strongly recommend a virtual environment. This keeps dependencies clean.
```bash
python -m venv .venv
source .venv/bin/activate

pip install -U pip
pip install -r requirements.txt
pip install -r requirements-dev.txt
```



## 6. Project Structure Overview

Key folders:
- `data/` → sample local data
- `jobs/` → pipeline entrypoints
- `src/project_a/` → core library + helpers
- `config/` and `local/config/` → configuration files
- `tests/` → test suite
```bash


# See the structure quickly
ls
ls jobs
ls src/project_a
```



## 7. Configuration (Local vs Prod)

Local configuration lives here:
- `local/config/local.yaml`

This file controls:
- Paths to data (`paths.bronze_root`, `paths.silver_root`, `paths.gold_root`)
- Which storage format to use (Iceberg/Delta/Parquet)
- Spark options

If you want local runs to be less strict, you can set:

```yaml


# local/config/local.yaml


# This prevents failure when data is incomplete


# (good for demos and learning)
dq:
  fail_on_error: false
```


## 8. How Spark is Built (Important)

This project automatically builds Spark with:
- Delta Lake support (if `delta-spark` is installed)
- Iceberg support (if enabled in config)

Local runs are forced to use **PySpark’s bundled Spark** (not Homebrew Spark 4), to avoid version conflicts.


## 9. Run the Pipeline Step by Step

### Step 1 — FX JSON → Bronze
This loads FX JSON and writes normalized Bronze data.
```bash
python3 -m project_a.pipeline.run_pipeline   --job fx_json_to_bronze   --env local   --config local/config/local.yaml
```

### Step 2 — Snowflake/CRM/Redshift → Bronze
These jobs simulate ingestion from different sources.
```bash
python3 -m project_a.pipeline.run_pipeline --job snowflake_to_bronze --env local --config local/config/local.yaml
python3 -m project_a.pipeline.run_pipeline --job crm_to_bronze --env local --config local/config/local.yaml
python3 -m project_a.pipeline.run_pipeline --job redshift_to_bronze --env local --config local/config/local.yaml
```

### Step 3 — Bronze → Silver
Cleans, standardizes, deduplicates.
```bash
python3 -m project_a.pipeline.run_pipeline --job bronze_to_silver --env local --config local/config/local.yaml
```

### Step 4 — Silver → Gold
Builds dimensional model + analytics tables.
```bash
python3 -m project_a.pipeline.run_pipeline --job silver_to_gold --env local --config local/config/local.yaml
```

### One‑Command Full Pipeline

Use the helper script:
```bash
python3 run_complete_etl.py --config local/config/local.yaml --env local
```



## 10. Screenshots of Outputs

Below are sample output “screenshots” generated from your local run.
![FX Bronze Output](/Users/kunal/IdeaProjects/Project_A/docs/interview/assets/fx_bronze_sample.svg)
![Gold Fact Orders Output](/Users/kunal/IdeaProjects/Project_A/docs/interview/assets/gold_fact_orders_sample.svg)


## 11. Validate Outputs

Check the Gold layer output:
```bash
ls data/gold
```

You can also load them in Spark and inspect:
```bash
from project_a.utils.spark_session import build_spark
from project_a.utils.config import load_config_resolved

config = load_config_resolved('local/config/local.yaml')
spark = build_spark(config)

df = spark.read.parquet('data/gold/fact_orders')
print(df.count())
df.show(5, False)

spark.stop()
```



## 12. Data Quality (DQ) Gates

The pipeline supports DQ checks (not‑null, uniqueness, range checks, contracts).

- For strict mode, set `dq.fail_on_error: true`.
- For learning/demo, keep it `false`.

DQ gate jobs:
```bash
python3 -m project_a.pipeline.run_pipeline --job dq_silver_gate --env local --config local/config/local.yaml
python3 -m project_a.pipeline.run_pipeline --job dq_gold_gate --env local --config local/config/local.yaml
```



## 13. Iceberg vs Delta vs Parquet (Beginner Explanation)

- **Parquet** = file format only (fast, simple, but not ACID)
- **Delta Lake** = Parquet + transaction log (ACID, time travel)
- **Iceberg** = table format with ACID + catalog integration

In this project:
- Local runs can use Parquet or Iceberg.
- If Iceberg is enabled, Silver/Gold use Iceberg tables.


## 14. Airflow (Optional)

You can orchestrate the pipeline using Airflow.

If you want to run locally with Docker:
```bash


# Example (if docker-compose-airflow.yml exists)
docker compose -f docker-compose-airflow.yml up -d
```

Open Airflow UI (default `http://localhost:8080`) and trigger the DAG.


## 15. dbt (Optional)

If you want dbt models:
```bash


# Make sure dbt profiles.yml exists
ls dbt



# Example command
cd dbt
python -m dbt debug
python -m dbt run
```



## 16. Tests

Run the test suite:
```bash
pytest -q
```



## 17. Common Errors + Fixes

**Error: Spark can’t parse timestamps**
- Fix: use robust parsing patterns in the extractor (already patched).

**Error: Table not found in Iceberg**
- This is normal on first run. The code falls back to path reads.

**Error: Missing DQGate import**
- Fixed by compatibility wrapper in `project_a.dq.gate`.

**Error: Spark 4.x mismatch**
- Local Spark is forced to PySpark distribution (Spark 3.5).


## 18. FAQ — Interview‑Style Q&A

**Q1: Why do you use Bronze/Silver/Gold layers?**  
A: It separates concerns. Bronze keeps raw truth, Silver standardizes, Gold optimizes for analytics.

**Q2: How do you guarantee data quality?**  
A: Contracts + DQ gates (not‑null, uniqueness, range checks) and fail‑fast in strict mode.

**Q3: Why Iceberg instead of only Parquet?**  
A: Iceberg adds ACID guarantees, time travel, schema evolution, and catalog integration.

**Q4: How do you avoid duplicate data?**  
A: Deduplication logic in Silver + merge/upsert strategies.

**Q5: How does this scale in production?**  
A: Spark on EMR with S3 storage, partitioned datasets, and orchestration via Airflow.

**Q6: What happens if a job fails halfway?**  
A: Jobs are idempotent, use overwrite/merge patterns, and produce run audits for recovery.

**Q7: How do you handle schema drift?**  
A: Explicit schemas + schema drift checks and contract validation.

**Q8: How do you trace lineage?**  
A: Lineage emitters record job inputs/outputs and metrics (OpenLineage‑style).

**Q9: How would you test this pipeline?**  
A: Unit tests for transforms + integration tests for stage‑level validation.


## 19. Interview‑Ready Summary

When asked to explain this project, say:

- It ingests multi‑source data into Bronze (raw).  
- Standardizes to Silver with explicit schema and data contracts.  
- Builds Gold fact/dim tables (analytics ready).  
- Includes DQ gates, lineage logging, and Iceberg/Delta support.  
- Designed for AWS EMR with S3 + Glue catalog.  

This gives you a strong, production‑grade story.
